In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [14]:
df_serie_hist = pd.read_csv(r'C:\Users\DuduCuco\Documents\NewDoencasRespiratorias\DoencasRespiratorias\Scripts\serie_hist.csv')

In [19]:
# Função para preparar a visão de diárias de UTI por mês/ano de um estabelecimento específico
def prepare_time_series(data):
    # Transformando as colunas 'Ano' e 'Mes' em uma única coluna de data
    data['Date'] = pd.to_datetime(data['Ano'].astype(str) + '-' + data['Mês'].astype(str) + '-01')
    
    # Definindo 'Date' como índice
    data.set_index('Date', inplace=True)
    
    # Selecionando apenas a coluna 'internacoes'
    data = data[['Internações']]
    
    return data

# Função de previsão usando SARIMAX com auto_arima
def forecast_time_series(data, steps=12):
    # Use auto_arima para encontrar os melhores hiperparâmetros do modelo SARIMA
    model = auto_arima(data, seasonal=True, m=12, stepwise=True, trace=True, error_action='ignore',
                      suppress_warnings=True, max_order=None)
    
    # Ajustar o modelo SARIMAX com os hiperparâmetros encontrados
    sarimax_model = SARIMAX(data, order=model.order, seasonal_order=model.seasonal_order)
    fitted_model = sarimax_model.fit(disp=False)
    
    # Fazer previsões
    forecast = fitted_model.get_forecast(steps=steps)
    forecast_index = pd.date_range(data.index[-1] + DateOffset(months=1), periods=steps, freq='M')
    forecast_values = forecast.predicted_mean
    forecast_conf_int = forecast.conf_int()
    
    # Construir dataframe de previsão
    forecast_df = pd.DataFrame({
        'Forecast': forecast_values,
        'Lower CI': forecast_conf_int.iloc[:, 0],
        'Upper CI': forecast_conf_int.iloc[:, 1]
    }, index=forecast_index)
    
    return forecast_df

# Função de previsão usando Prophet
def forecast_time_series_prophet(data):
    # Preparar o dataframe para o Prophet
    data_prophet = pd.DataFrame({
        'ds': data.index,
        'y': data.values
    })
    
    # Ajustar o modelo Prophet
    prophet_model = Prophet()
    prophet_model.fit(data_prophet)
    
    # Fazer previsões
    future = prophet_model.make_future_dataframe(periods=12, freq='M')
    forecast = prophet_model.predict(future)
    
    return forecast.set_index('ds')['yhat']

# Função para calcular o MSE
def calculate_mse(actual, predicted):
    return mean_squared_error(actual, predicted)

# Carregar o dataframe df_serie_hist
data = df_serie_hist

# Preparar a série temporal
time_series = prepare_time_series(data)

# Previsão com SARIMAX
forecast_sarimax = forecast_time_series(time_series)

# Previsão com Prophet
forecast_prophet = forecast_time_series_prophet(time_series)

# Calcular o MSE para cada previsão
mse_sarimax = calculate_mse(time_series[-12:], forecast_sarimax['Forecast'])
mse_prophet = calculate_mse(time_series[-12:], forecast_prophet[-12:])

# Imprimir os MSEs
print(f"MSE do modelo SARIMAX: {mse_sarimax}")
print(f"MSE do modelo Prophet: {mse_prophet}")

# Comparar os MSEs e escolher o melhor modelo
if mse_sarimax < mse_prophet:
    print("O modelo SARIMAX é o melhor.")
    best_forecast = forecast_sarimax['Forecast']
else:
    print("O modelo Prophet é o melhor.")
    best_forecast = forecast_prophet[-12:]

# Plotar as previsões
plt.figure(figsize=(10, 6))
plt.plot(time_series, label='Dados originais')
plt.plot(forecast_sarimax['Forecast'], label=f'Previsão SARIMAX (MSE: {mse_sarimax:.2f})')
plt.plot(forecast_prophet[-12:], label=f'Previsão Prophet (MSE: {mse_prophet:.2f})')
plt.legend()
plt.title("Previsões para internações")
plt.show()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=195953.116, Time=61.30 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=200537.468, Time=0.63 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=197684.016, Time=5.57 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=196686.385, Time=12.70 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=200535.470, Time=0.32 sec


: 